# ***Đồ án môn học Nhập môn Phân tích dữ liệu và Học sâu***

In [7]:
import pandas as pd

def load_data():
    df = pd.read_csv("vietnam_housing_dataset.csv")
    return df
df = load_data()

def load_10(df):
    print(df.head(10))
load_10(df)

                                             Address  Area  Frontage  \
0  Dự án The Empire - Vinhomes Ocean Park 2, Xã L...  84.0       NaN   
1  Dự án The Crown - Vinhomes Ocean Park 3, Xã Ng...  60.0       NaN   
2  Dự án The Crown - Vinhomes Ocean Park 3, Xã Ng...  90.0       6.0   
3  Đường Nguyễn Văn Khối, Phường 11, Gò Vấp, Hồ C...  54.0       NaN   
4   Đường Quang Trung, Phường 8, Gò Vấp, Hồ Chí Minh  92.0       NaN   
5  Dự án The Empire - Vinhomes Ocean Park 2, Xã L...  91.0       7.0   
6  Dự án The Empire - Vinhomes Ocean Park 2, Xã L...  64.0       4.0   
7  Dự án Him Lam Thường Tín, Huyện Thường Tín, Hà...  74.0       5.0   
8  Dự án The Empire - Vinhomes Ocean Park 2, Xã L...  48.0       NaN   
9  Dự án The Crown - Vinhomes Ocean Park 3, Xã Ng...  91.0       7.0   

   Access Road House direction Balcony direction  Floors  Bedrooms  Bathrooms  \
0          NaN             NaN               NaN     4.0       NaN        NaN   
1          NaN             NaN               

# **Mô tả dữ liệu**